In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras.models import Sequential 
#from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Activation , Dense 
from tensorflow.keras.applications import MobileNet

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
!mkdir train
!mkdir test

In [ ]:
!tar -xf /content/drive/My\ Drive/Colab\ Notebooks/Train.tar.gz -C train

In [ ]:
!tar -xf /content/drive/My\ Drive/Colab\ Notebooks/Test.tar.gz -C test


In [ ]:
train_datagen=ImageDataGenerator(preprocessing_function=lambda x: (x / 127.5)-1, horizontal_flip=True)


import pandas as pd
df = pd.read_csv('train/testset.csv')
df['Label'] = df['Label'].astype(str)


train_generator = train_datagen.flow_from_dataframe(
    df, directory='train/testset', x_col='Data', y_col='Label',
    target_size=(224, 224),batch_size=32, 
)

Found 9600 validated image filenames belonging to 48 classes.


In [ ]:
test_datagen=ImageDataGenerator(preprocessing_function=lambda x: (x / 127.5)-1)

In [ ]:
import pandas as pd
df = pd.read_csv('test/testset.csv')
df['Label'] = df['Label'].astype(str)


test_generator = test_datagen.flow_from_dataframe(
    df, directory='test/testset', x_col='Data', y_col='Label',
    target_size=(224, 224),batch_size=32, 
)

Found 2609 validated image filenames belonging to 48 classes.


In [ ]:
df.loc[df['Data']=='100037.jpeg']

,Data,Label
936,100037.jpeg,43


In [ ]:
base_model=MobileNet(input_shape=(224, 224, 3), include_top = False, weights = 'imagenet')

17227776/17225924 [==============================] - 0s 0us/step


In [ ]:
for layers in base_model.layers[0:20]:
  layers.trainable = False
for layers in base_model.layers[20:]:
  layers.trainable = True

In [ ]:
global_average_layer = GlobalAveragePooling2D()
prediction_layer = Dense(48,activation ='softmax')

In [ ]:
model = Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [ ]:

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenet_1.00_224 (Model)   (None, 7, 7, 1024)        3228864   
_________________________________________________________________
global_average_pooling2d (Gl (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 48)                49200     
Total params: 3,278,064
Trainable params: 3,242,160
Non-trainable params: 35,904
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(train_generator, validation_data=test_generator,
                   epochs=1)

300/300 [==============================] - 82s 275ms/step - loss: 0.7429 - accuracy: 0.8218 - val_loss: 0.0929 - val_accuracy: 0.9904


In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
img = load_img('/content/test/testset/100037.jpeg', target_size = (224,224))

In [ ]:
import numpy as np
img = np.array(img)

In [ ]:
img = (img/127.5)-1

In [ ]:
img = np.expand_dims(img, axis = 0)

In [ ]:
img.shape

(1, 224, 224, 3)

In [ ]:
model(img, training=False) #43 expectation

<tf.Tensor: shape=(1, 48), dtype=float32, numpy=
array([[1.3948813e-05, 2.3868419e-05, 7.9966494e-06, 4.2693118e-06,
        2.4108526e-06, 2.0492471e-06, 7.7858937e-07, 6.0611859e-07,
        1.0101489e-06, 3.5442630e-05, 6.3465944e-07, 4.9013364e-05,
        1.8023014e-05, 2.1652909e-06, 1.5674257e-06, 4.2538431e-06,
        6.7308787e-07, 2.1669105e-06, 7.3386462e-07, 1.2068440e-06,
        5.2635488e-07, 2.2009187e-06, 1.0448822e-05, 6.4248334e-06,
        2.6132158e-05, 1.8349091e-04, 3.5179357e-05, 2.7175454e-05,
        4.3618861e-06, 2.9361247e-06, 3.4146219e-06, 3.4849447e-05,
        5.9895774e-05, 5.5056021e-06, 2.8422719e-06, 4.5466654e-06,
        6.8639318e-04, 7.6589745e-04, 9.9722445e-01, 2.3465647e-04,
        3.7895945e-05, 3.5592038e-05, 2.0962534e-06, 8.7265427e-05,
        7.6806209e-06, 2.5855255e-04, 5.5868786e-05, 2.0875304e-05]],
      dtype=float32)>

In [ ]:
model.evaluate(test_generator)

82/82 [==============================] - 15s 180ms/step - loss: 0.0929 - accuracy: 0.9904


[0.09289073199033737, 0.9904177784919739]